In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn.preprocessing as pre
from sklearn import model_selection

sns.set_palette(sns.color_palette("Set1", n_colors=8, desat=1))

In [185]:
class NN(object):

    def __init__(self, n_nodes):
        self.n_nodes = n_nodes
        self.weights, self.bias = self.init_weights()

    # get final output of nn from input X based on current weights and biases
    # Assumes X a numpy representation  of the data in wide format. 
    def feed_foward(self, X, train=False, debug=False):
        N = len(self.n_nodes)
        iters = np.arange(1, N)
        activations = [X.T]
        zs = []
        for i, b, w in zip(iters, self.bias, self.weights):  
            z = w @ activations[i - 1] + b
            a = self.sigmoid(z)
            activations.append(a)
            zs.append(z)
            if debug:
                print(i)
                print(w.shape, activations[i-1].shape, b.shape, z.shape)
            
        if train:
            return activations, zs
        return activations[-1]
    
    def back_prop(X, Y):
        
        # Perform feed forward 
        A, Z = feed_forward(X, train=True)
        
        # Compute output layer error:
        

    # helper functions --------------------
    def sigmoid(self, X):
        return 1 / (1 + np.exp(-X))

    def probability(self, X, weights, bias):
        return self.sigmoid(np.dot(X, weights) + bias)

    def init_weights(self):
        weights = [np.random.randn(m, n) for m, n in zip(self.n_nodes[1:], self.n_nodes[:-1])]
        bias = [np.zeros(m).reshape(-1,1) for m in self.n_nodes[1:]]  
        return weights, bias
    # ------------------------------------
        

In [203]:
np.random.seed(44)
net = NN((2, 3, 2))

In [204]:
for w, b in zip(net.weights, net.bias):
    print(f'w=\n{w}')
    print(f'b={b}\n ---------------')

w=
[[-0.75061472  1.31635732]
 [ 1.24614003 -1.60491574]
 [-1.46814368 -1.71507046]]
b=[[0.]
 [0.]
 [0.]]
 ---------------
w=
[[ 1.85878369  0.08758798 -0.0523222 ]
 [ 0.55547164 -0.96340369 -0.18032147]]
b=[[0.]
 [0.]]
 ---------------


In [205]:
x = np.arange(4).reshape(2,2)
net.feed_foward(x)

array([[0.81343161, 0.84795289],
       [0.56201834, 0.60472459]])